<a href="https://colab.research.google.com/github/madhushalini06785/RAG-chatbot/blob/main/YardstickTask1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain langchain-community sentence-transformers pinecone-client==3.0.0 transformers
import os
os.environ["PINECONE_API_KEY"] = "pcsk_GnFYB_GrhvMT43cMX8oFHjKgQSwaJnX36cTmD3A1FyV3wFWbXJXfRNJ1PLSEyrQTDYhS5"
os.environ["PINECONE_ENV"] = "us-east-1"  # Example: "us-east-1"

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Business_info.txt to Business_info (1).txt


In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
loader = TextLoader("Business_info.txt")
documents = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(documents)
print(f"✅ Total Chunks: {len(split_docs)}")

✅ Total Chunks: 2


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
texts = [doc.page_content for doc in split_docs]
metadatas = [doc.metadata for doc in split_docs]
embeddings = embedding_model.embed_documents(texts)
print("✅ HuggingFace embeddings created.")

✅ HuggingFace embeddings created.


In [ ]:
from pinecone import Pinecone
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
index_name = "index"
index = pc.Index(index_name)
all_vectors = [
    {
        "id": f"chunk-{i}",
        "values": embeddings[i],
        "metadata": {**metadatas[i], "text": texts[i]}
    }
    for i in range(len(embeddings))
]
index.upsert(vectors=all_vectors)
print("✅ All vectors uploaded to Pinecone index.")

✅ All vectors uploaded to Pinecone index.


In [ ]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Pinecone as LangchainPinecone
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
vectorstore = LangchainPinecone.from_existing_index(
    index_name=index_name,
    embedding=embedding_model
)
retriever = vectorstore.as_retriever()
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-small", max_length=512)
llm = HuggingFacePipeline(pipeline=qa_pipeline)
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
query = "What services does the company offer?"
response = qa_chain.invoke(query)
print("\nAnswer:",response)

Device set to use cpu



Answer: {'query': 'What services does the company offer?', 'result': 'Cloud Computing Solutions (AWS, Azure, GCP) - DevOps and CI/CD Pipelines - AI & Machine Learning Consulting - IT Infrastructure Management'}
